In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.formula.api as smf

from scipy.optimize import minimize

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings('ignore')


## modelagem GLM

In [6]:
%%time
df_mod_mp_feb23may24_v3 = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Renovacao/df_mod_mp_feb23may24_v3.csv",
                     encoding='latin-1',
                     delimiter='|')

CPU times: total: 5.97 s
Wall time: 7.04 s


In [7]:
df_mod_mp_feb23may24_v3.shape

(571594, 105)

###  Prepara base de modelagem glm

In [8]:
df_mod_filtered=df_mod_mp_feb23may24_v3[["premio_ofertado","premio_total", 'premio_creserva','premio_demais_cob',"teste_validacao","ind_renovacao","MD_sexo","cod_corr",
                                "Cod_End","MD_sin_5a_cas","MD_tempo_casa_2","MD_idade_veic","MD_faixa_IScas",
                                "anomes_dt_ter" ,"MD_idade","per_com", "RNS_Casco_calc","MD_menor26","UF","Renovacao_ant",
                                "fx_ir_12m","is_casco_ofer","classe_bonus","MD_fator_ajuste","MD_capac","md_categoria",
                                "MD_comb","MD_qtd_assis_cpf", "idade_ult_RNS","canal","MD_tipo_renov",'FABRICA','MD_est_civil',
                                'peso','Grupo_veiculo_new','vigencia',
                                'Risco_PP_mod', 'Risco_PT_mod','Risco_RF_mod','Risco_DM_mod','Risco_AS_mod',
                                'Risco_DC_mod' ,'Risco_DMO_mod']]

In [9]:
df_mod_filtered = df_mod_filtered[df_mod_filtered['per_com'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['Renovacao_ant'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['MD_est_civil'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['peso'].notna()]
df_mod_filtered = df_mod_filtered.query('premio_total !=0')
df_mod_filtered = df_mod_filtered.query('vigencia == "Anual"')
df_mod_filtered = df_mod_filtered.query('premio_creserva >= 0')
df_mod_filtered = df_mod_filtered[~df_mod_filtered['anomes_dt_ter'].isin([202310,202309,202308,202302,202402])]
df_mod_filtered = df_mod_filtered.query('is_casco_ofer > 0')
df_mod_filtered = df_mod_filtered[(df_mod_filtered['per_com']>0) & (df_mod_filtered['per_com']<50)]
df_mod_filtered = df_mod_filtered.query('premio_total >100')
df_mod_filtered = df_mod_filtered.query('MD_idade !="ERRO"')
df_mod_filtered = df_mod_filtered[(df_mod_filtered.UF != "SP") | (~df_mod_filtered.Grupo_veiculo_new.isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total <=2.3) &  (df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total>=0.7)]

df_mod_filtered.shape


(358297, 43)

In [10]:
df_mod_filtered['DA_emissao'] =0.2349
df_mod_filtered['DA_producao'] =0.2749
df_mod_filtered['risco_dmais_cob'] =df_mod_filtered['premio_demais_cob']*0.2627*1.4
df_mod_filtered['premio_anterior'] =df_mod_filtered['premio_total']
df_mod_filtered['Act_cost']  = df_mod_filtered['Risco_PP_mod'] + df_mod_filtered['Risco_PT_mod'] + df_mod_filtered['Risco_RF_mod']+ df_mod_filtered['Risco_DM_mod']+ df_mod_filtered['Risco_AS_mod'] + df_mod_filtered['Risco_DC_mod'] +df_mod_filtered['Risco_DMO_mod'] + df_mod_filtered['risco_dmais_cob']
df_mod_filtered['Premio_IC100']  = df_mod_filtered['Act_cost']/(1-df_mod_filtered['per_com']/100-df_mod_filtered['DA_producao'] )
df_mod_filtered["premio"] = df_mod_filtered["premio_ofertado"]

In [11]:
def categorizar(df):
    
    df['classe_bonus_bin'] = df['classe_bonus'].apply(lambda x: '00'  if x == 0 else 'Demais')   
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)
    
    df['MD_sexo_2bin'] = df['MD_sexo'].map({'Masculino': 1, 
                                 'Feminino': 0, 
                                  'ERRO': 1}).astype(int)


    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01 - 00-<=020' if x == '01 - 00-<=020' else
                                                              '02 - 20-<=040' if x == '02 - 20-<=040' else
                                                              '03 - 40-<=060' if x == '03 - 40-<=060' else
                                                              '04 - 60-<=080' if x == '04 - 60-<=080' else
                                                              '05 - 80-<=100' if x == '05 - 80-<=100' else
                                                              '02 - 20-<=040' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor Mais' if x == 'Corretor Mais'  else 'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: '<10' if x < 10 else
                                                          '>=10' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x: 'Sem Sinistro' if x == 'Sem Sinistro' else
                                                                      '01-05 ano' if x == '01 ano' else
                                                                      '01-05 ano' if x == '01 ano' else
                                                                      '01-05 ano' if x == '01 ano' else
                                                                      '01-05 ano' if x == '01 ano' else
                                                                      '01-05 ano' if x == '01 ano' else '06+ ano +')


    df['UF_bin'] = df['UF'].apply(lambda x: 'AL,CE,RS'  if x == 'AL' else
                                                'AL,CE,RS'  if x == 'CE' else
                                                'AL,CE,RS'  if x == 'RS' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'AP' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'DF' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'ES' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'PA' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'PB' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'RN' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'SC' else 
                                                'GO,MA,MG'             if x == 'GO' else 
                                                'GO,MA,MG'             if x == 'MA' else 
                                                'GO,MA,MG'             if x == 'MG' else 
                                                'MT,PR'                if x == 'MT' else
                                                'MT,PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x:     '1 - SeguroNovo'       if x == '1 - Seguro Novo'  else
                                                                          '2 - RenovPropria'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                          '2 - RenovPropria'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                          '3 - RenovCongenere'   if x == '4 - Renov Congenere sem sinistro' else
                                                                          '3 - RenovCongenere'   if x == '5 - Renov Congenere com sinistro' else
                                                                          '3 - RenovCongenere'   if x == '6 - Renov BB sem sinistro' else
                                                                          '3 - RenovCongenere'   if x == '7 - Renov BB com sinistro' else  '2 - RenovPropria')
       
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                      '13509'  if x == 50007 else
                                                      '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: '<101'    if x < 101 else
                                                                    '>=101'   if x >= 101  else '<101')   

    df['MD_qtd_assis_cpf_bin'] = df['MD_qtd_assis_cpf'].apply(lambda x: '00'  if x == 0 else
                                                                        '01'  if x == 1 else
                                                                        '>=2' if x >= 2 else 'ERROR')
     
    return df

In [12]:
df_mod=df_mod_filtered.copy()
df_mod = categorizar(df_mod)

In [13]:
df_train = df_mod.query('teste_validacao == "Trein"')
df_train.shape

(286808, 64)

In [14]:
df_test = df_mod.query('teste_validacao == "Teste"')
df_test.shape

(71489, 64)

In [15]:
#x_df_num = df_train[['premio']]
#x_df_num = x_df_num.fillna(0)
#x_df_cat = df_train[['MD_sexo_bin']]
#y   = df_train[['ind_renovacao']]
#x_df_dummy=pd.get_dummies(x_df_cat)
#x_df=pd.concat([x_df_num, x_df_dummy],axis=1)
#x_train, x_test, y_train, y_test = train_test_split(x_df, y, test_size=0.2) 
#x_train = x_df

In [16]:
#Modelo GLM de retenção
#formula = 'ind_renovacao ~ 1 + MD_sexo_bin_Feminino  + np.log(premio) ' 
#model_glm = smf.logit(formula , data=x_df).fit()
#model_glm.summary()

In [18]:
import statsmodels.formula.api as smf
#Modelo GLM de retenção
#formula = 'ind_renovacao ~ 1 + MD_sexo_2bin + np.log(premio) ' 
formula = 'ind_renovacao ~ 1 + MD_sexo_2bin + premio '
model_glm = smf.logit(formula , data=df_train).fit()
model_glm.summary()

Optimization terminated successfully.
         Current function value: 0.651262
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          ind_renovacao   No. Observations:               286808
Model:                          Logit   Df Residuals:                   286805
Method:                           MLE   Df Model:                            2
Date:                Sat, 10 Aug 2024   Pseudo R-squ.:                 0.05603
Time:                        20:52:09   Log-Likelihood:            -1.8679e+05
converged:                       True   LL-Null:                   -1.9787e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.7437      0.009     85.725      0.000       0.727       0.761
MD_sexo_2bin     0.0838      0.008     10.723      0.000       0.068       0.099
premio          -0.0003   2.67e-06   -125.884      0.000      -0.000      -0.000
================================================================================
"""

In [19]:
df_train["prob_est"]=model_glm.predict(df_train)


## Base Para escorar e otimizar

In [20]:
df_lote_092024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Table 7 Base Case Total  - teste novo.csv",
                     encoding='latin-1',
                     delimiter='|')

In [174]:
#df_lote_092024.groupby(['Amostra'])['GBM_mercado',"Renewal Demand"].describe().T

In [21]:
df_lote_escorar=df_lote_092024.copy()
df_lote_escorar.shape

(27304, 123)

In [28]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [29]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'Classe_bonus':'classe_bonus',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [30]:
df_lote = categorizar(df_lote_escorar)
df_lote.head(5)

,Act_cost,Amostra,Amostra_inicial,ano_fab,ano_modelo,batente_max_ti,batente_min_ti,batente_min_ti_original,canal,Carprice,...,canal_bin,per_com_bin,idade_ult_RNS_bin,UF_bin,md_tipo_renov_ant_bin,Cod_End_bin,MD_capac_bin,fator_ajuste_bin,MD_qtd_assis_cpf_bin,premio
0,1089.960657,Otimização Mkp,Otimização Mkp,2019,2019,2.3,0.5,0.6,BancoVotorantim,49491,...,Demais,>=10,Sem Sinistro,"AP,DF,ES,PA,PB,RN,SC",1 - SeguroNovo,Demais,Demais,<101,00,2012.730000
1,201.505993,Otimização Atu,Otimização Atu,2008,2008,2.3,0.5,0.6,MNP,15989,...,Demais,<10,Sem Sinistro,Demais,2 - RenovPropria,Demais,Demais,<101,00,281.787153
2,233.955581,Otimização Mkp,Otimização Mkp,2019,2020,2.3,0.5,0.6,Coamo,77960,...,Demais,>=10,Sem Sinistro,Demais,3 - RenovCongenere,Demais,Demais,<101,00,374.269047
3,304.632691,Controle,Controle,2022,2023,2.3,0.5,0.6,MNP,124965,...,Demais,<10,Sem Sinistro,Demais,2 - RenovPropria,Demais,Demais,<101,00,426.000127
4,198.441307,Otimização Mkp2,Otimização Mkp,1993,1993,2.3,0.5,0.6,Corretor,8021,...,Demais,>=10,Sem Sinistro,"AL,CE,RS",2 - RenovPropria,Demais,Demais,<101,00,369.890000


In [31]:
df_lote["prob_est"]=model_glm.predict(df_lote)

In [209]:
#df_lote.groupby(["Amostra"])["prob_est" , "Renewal Demand"].describe().T

## Otimização Teste 

In [182]:
#print(df_lote['Amostra'].value_counts(ascending=False))

In [32]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra == "Otimização Atu"')
df_lote_otim_atu.shape

(10925, 139)

In [33]:
#df_lote_otim = df_lote_otim_atu

In [247]:
df_lote_otim = df_lote_otim_atu.sample(n = 5) 
df_lote_otim.shape

(5, 139)

In [51]:
model_glm.params[0]

0.7437110315116003

In [52]:
model_glm.params[1]

0.08379804132383697

In [53]:
model_glm.params[2]

-0.0003355330181045059

In [248]:
df_lote["prob_est"]=model_glm.predict(df_lote)

In [194]:
#df_lote_otim["Margem Inicial"] = df_lote_otim["prob_est"]*(df_lote_otim["Premio_oferecido_com_batente"]-df_lote_otim["Act_cost"])

In [249]:
df_teste = df_lote_otim[['Act_cost','premio','prob_est','MD_sexo_2bin']]
df_teste

,Act_cost,premio,prob_est,MD_sexo_2bin
3632,692.105159,1318.044484,0.574800,0
26998,5921.704558,6458.866998,0.194121,0
3431,736.208140,1402.034165,0.567899,0
3423,673.578481,1417.761485,0.566603,0
7532,850.037294,1789.175530,0.535785,0


In [350]:
def demanda1(X, P):
    # X é uma variável indicadora (X=1/0)
    #D = np.exp(0.7437110315116003 + 0.08379804132383697 * X - 0.0003355330181045059 * P) / (1 + np.exp(0.7437110315116003 + 0.08379804132383697 * X - 0.0003355330181045059 * P))
    D = np.exp(0.64 + 0.4 * X - 0.005 * P) / (1 + np.exp(0.64 + 0.4 * X - 0.005 * P))
    return D

In [351]:
demanda1(1,250)


0.4476920904256747

In [352]:
demanda1(0,300)

0.2973393456552685

In [353]:
(demanda1(1,250)+demanda1(0,300))/2

0.37251571804047157

array([-2.59849124,  1.70894129, -0.12903317, -1.33290873])

In [326]:
# Criação do DataFrame
df = pd.DataFrame({
    'Cliente': [1, 2, 3, 4, 5,6],
    'X': [1, 1, 1, 0, 0,0],
    'P': [250, 250, 250, 300, 300,300],
    'C': [150, 150, 150, 150, 150,150],
    'D': [0.447, 0.447, 0.447, 0.297, 0.297,0.297]
})


df['margem']=df['D']*(df['P'] - df['C'])
df

,Cliente,X,P,C,D,margem
0,1,1,250,150,0.447,44.70
1,2,1,250,150,0.447,44.70
2,3,1,250,150,0.447,44.70
3,4,0,300,150,0.297,44.55
4,5,0,300,150,0.297,44.55
5,6,0,300,150,0.297,44.55


In [295]:
# Calcular a média ponderada
#media_ponderada = (df['Notas'] * df['Pesos']).sum() / df['Pesos'].sum()

# Exibir o resultado
#print("Média Ponderada:", media_ponderada)

In [165]:
#df_teste["D"]=demanda1(df_teste['MD_sexo_2bin'],df_teste['premio'])
#df_teste

In [296]:
#df["D"]=demanda1(df['X'],df['P'])
#df=df[0:1,3:4]
#df

In [327]:
df = df.iloc[[0,4], :]
df

,Cliente,X,P,C,D,margem
0,1,1,250,150,0.447,44.70
4,5,0,300,150,0.297,44.55


In [343]:
# Exemplo de uso
#custo_medio=np.mean(df['C'])
custo_medio=np.array(df['C'])
target_demand = np.mean(df['D'])
indicadora = np.array(df['X'])  # Exemplo de variáveis indicadoras para dois clientes
precos_iniciais = np.array(df['P']*1.7) # Exemplo de preços iniciais para dois clientes

In [344]:
custo_medio,target_demand,indicadora,precos_iniciais

(array([150, 150], dtype=int64),
 0.372,
 array([1, 0], dtype=int64),
 array([425., 510.]))

In [256]:
# Exemplo de uso
custo_medio=np.array(df_teste['Act_cost'])
#custo_medio=np.mean(df_teste['Act_cost'])
target_demand = np.mean(df_teste['prob_est'])
indicadora = np.array(df_teste['MD_sexo_2bin'])  # Exemplo de variáveis indicadoras para dois clientes
precos_iniciais = np.array(df_teste['premio']*1) # Exemplo de preços iniciais para dois clientes

In [217]:
#custo_medio

In [362]:
indicadora = [1, 0]  # Exemplo de variáveis indicadoras para dois clientes
precos_iniciais = [400, 500]  # Exemplo de preços iniciais para dois clientes
target_demand=0.7450314360809431/2
#target_demand=0.6666155296440016/2
custo_medio = 150

In [363]:
%%time
precos_otimizados, lambida = otimizar_precos(indicadora, precos_iniciais)
print("Preços Otimizados:", precos_otimizados)
print("Multiplicador de Lagrange (λ):", lambida)


Preços Otimizados: [419.47373108 500.        ]
Multiplicador de Lagrange (λ): 0.0
CPU times: total: 0 ns
Wall time: 998 µs


In [213]:
df_teste['Preços Otimizados']=precos_otimizados
df_teste["D_otimizada"]=demanda1(df_teste['MD_sexo_2bin'],df_teste['Preços Otimizados'])
df_teste

,Act_cost,premio,prob_est,MD_sexo_2bin,Preços Otimizados,D_otimizada
24112,3387.573658,5367.210000,0.274210,1,37903.931000,0.000007
14219,1291.506246,2459.543412,0.479625,0,2705.497754,0.459074
23099,3125.039560,4999.263414,0.299452,1,5499.189755,0.265485
18620,1790.983676,3131.865000,0.423815,0,3445.051500,0.398381
19691,1703.737066,2725.543218,0.478264,1,2998.097540,0.455505
...,...,...,...,...,...,...
14955,1393.678276,2556.738719,0.471491,0,2812.412591,0.450179
7395,1074.892216,1719.552417,0.541590,0,1891.507659,0.527235
4295,840.975185,1389.811908,0.568905,0,1528.793099,0.557433
13414,1762.627745,2819.753232,0.470382,1,3101.728555,0.446895


In [337]:
#df_teste[['D','D_otimizada']].describe()

In [365]:
import numpy as np
import sympy as sp
from scipy.optimize import fsolve

# Define a função de demanda
def demanda(X, P):
    # X é uma variável indicadora (X=1/0)
    #D = np.exp(0.7437110315116003 + 0.08379804132383697 * X - 0.0003355330181045059 * P) / (1 + np.exp(0.7437110315116003 + 0.08379804132383697 * X - 0.0003355330181045059 * P))
    D = np.exp(0.64 + 0.4 * X - 0.005 * P) / (1 + np.exp(0.64 + 0.4 * X - 0.005 * P))
    return D

# Define a função de Lagrange
def func(X, indicadora):
    P = X[:-1]  # Extraindo o vetor de preços [p1, p2, ..., pn]
    L = X[-1]   # Extraindo o multiplicador de Lagrange
    #L =  np.zeros(len(P))   # Extraindo o multiplicador de Lagrange
    n = len(P)
    
    # Função objetivo e a soma das demandas
    objetivo = 0
    soma_demandas = 0
    
    for i in range(n):
        D_i = demanda(indicadora[i], P[i])
        objetivo += D_i * (P[i] - custo_medio)
        soma_demandas += D_i
    
    # Definindo a função de Lagrange com a restrição
    return objetivo - L * (soma_demandas - target_demand*n)

# Aplicando a diferenciação numérica
def dfunc(X, indicadora):
    dLambda = np.zeros(len(X))
    h = 1e-3  # Passo usado na diferença finita
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = h
        dLambda[i] = (func(X + dX, indicadora) - func(X - dX, indicadora)) / (2 * h)
        return dLambda

  
    
    
# Definindo a função para otimizar os preços
def otimizar_precos(indicadora, precos_iniciais):
    # Iniciando o vetor X com os preços iniciais e um valor inicial para o multiplicador de Lagrange
    X_inicial = np.append(precos_iniciais, 0)
    
# Calculando os preços otimizados
   X_otimizado = fsolve(dfunc, 
  #                       X_inicial, 
   #                      args=(indicadora))
    
    # Retornando o vetor de preços otimizados e o multiplicador de Lagrange
    #precos_otimizados = X_otimizado[:-1]
    #lambda_ = X_otimizado[-1]
    
    #return precos_otimizados, lambda_



In [366]:
print(X1) 

[419.47373108 500.           0.        ]


In [107]:
# Definindo a nova função de demanda
def demanda(P, X):
    return np.exp(0.7437 + 0.0837 * X - 0.000335 * P) / (1 + np.exp(0.7437 + 0.0837 * X - 0.000335 * P))


In [367]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Criação do DataFrame
#df = pd.DataFrame({
#    'Cliente': [1, 2, 3, 4, 5],
#    'X': [1, 1, 1, 0, 0],
#    'P': [1870, 6160, 4290, 2521, 2361],
#    'C': [1079, 3850, 2467, 1576, 1358],
#    'D': [0.549, 0.224, 0.351, 0.474, 0.487]
#})

df = pd.DataFrame({
    'Cliente': [1, 2],
    'X': [1, 0],
    'P': [250, 300],
    'C': [150, 150],
    'D': [0.474, 0.297]
})

# Definindo a nova função de demanda
def demanda(P, X):
    #return np.exp(0.7437 + 0.0837 * X - 0.000335 * P) / (1 + np.exp(0.7437 + 0.0837 * X - 0.000335 * P))
    return np.exp(0.64+0.4*X - 0.005*P)/((1+np.exp(0.64+0.4*X - 0.005*P)))

# Função de Lagrange com as restrições
def func(X, df):
    P = X[:-1]  # Extraindo o vetor de preços otimizados [p1, p2, ..., pn]
    L = X[-1]   # Extraindo o multiplicador de Lagrange
    n = len(P)
    
    margem_total = 0
    soma_demandas = 0
    
    for i in range(n):
        D_i = demanda(P[i], df['X'][i])
        margem_total += D_i * (P[i] - df['C'][i])
        soma_demandas += D_i
    
    return margem_total - L * (soma_demandas - sum(df['D']))

# Restrições para os preços otimizados
def constraint(X, df):
    P = X[:-1]
    L = X[-1]
    precos_iniciais = df['P'].values
    n = len(P)
    
    constraints = []
    
    # Restrições para garantir que os preços fiquem entre 90% e 110% dos preços iniciais
    for i in range(n):
        constraints.append({'type': 'ineq', 'fun': lambda X: X[i] - 0.9 * precos_iniciais[i]})
        constraints.append({'type': 'ineq', 'fun': lambda X: 1.1 * precos_iniciais[i] - X[i]})
    
    return constraints

# Função para otimizar os preços
def otimizar_precos(df):
    precos_iniciais = df['P'].values
    X_inicial = np.append(precos_iniciais, 0)  # Adicionando um valor inicial para o multiplicador de Lagrange
    
    # Executando a otimização com restrições
    resultado = minimize(func, X_inicial, args=(df,), method='SLSQP', constraints=constraint(X_inicial, df))
    
    precos_otimizados = resultado.x[:-1]
    lambida = resultado.x[-1]
    
    return precos_otimizados, lambida

# Executar otimização para os preços
precos_otimizados, lambida = otimizar_precos(df)
precos_otimizados, lambida


(array([-141748.54257849,     278.27798705]), -3151.921444871386)

In [46]:
 df_teste['MD_sexo_2bin']

5501     1
25342    1
22067    1
13579    0
17593    0
Name: MD_sexo_2bin, dtype: int32

#########################################

In [188]:
# premio otimizados
premio_otim = result.x
df_lote_otim["premio"]=result.x
df_lote_otim["demanda otimizada"] = model_glm.predict(df_lote_otim)
df_lote_otim["Margem otimizada"] = df_lote_otim["demanda otimizada"]*(df_lote_otim["premio"]-df_lote_otim["Act_cost"])

In [189]:
print(np.mean(df_lote_otim["Premio_oferecido_com_batente"]))
print(np.mean(df_lote_otim["premio"]))
print(np.mean(df_lote_otim["Margem Inicial"]))
print(np.mean(df_lote_otim["Margem otimizada"]))
print(np.mean(df_lote_otim["prob_est"]))
print(np.mean(df_lote_otim["demanda otimizada"]))


2981.7841009168
2981.860280769805
426.8343062015792
426.8754092048532
0.4698531952629786
0.46983657231791


In [190]:
prices_df = pd.DataFrame({
    'chave': df_lote_otim["chave"],
    'Act_cost':  df_lote_otim["Act_cost"],
    'Premio ofertado com batente':  df_lote_otim["Premio_oferecido_com_batente"],
    'Optimized Prices': df_lote_otim["premio"],
    'lb_bound' : [b[0] for b in bounds],
    'up_bound' : [b[1] for b in bounds],
    'prob_est':         df_lote_otim["prob_est"],
    'renewal demand':df_lote_otim["demanda otimizada"],
    'variação':df_lote_otim["premio"]/df_lote_otim["Premio_oferecido_com_batente"]
})
prices_df.head(5)

,chave,Act_cost,Premio ofertado com batente,Optimized Prices,lb_bound,up_bound,prob_est,renewal demand,variação
7261,56509736733100001,899.821476,1713.619265,1713.697119,1542.257339,1884.981192,0.559198,0.559184,1.000045
3416,605401733543100001,660.122232,1257.136226,1257.307940,1131.422603,1382.849849,0.652000,0.651961,1.000137
6635,168100190953100001,1018.497275,1802.331048,1802.415450,1622.097943,1982.564153,0.565637,0.565623,1.000047
25028,135101190983100001,3631.597126,5809.625862,5809.620041,5228.663276,6390.588448,0.214343,0.214344,0.999999
25387,347200137683100001,2754.524955,3735.075000,3735.090280,3361.567500,4108.582500,0.342273,0.342272,1.000004


### Aplicando processo de Lagrange

In [191]:
df = df_lote_otim[["premio", "MD_sexo_2bin","prob_est"]]
df.head(5)

,premio,MD_sexo_2bin,prob_est
7261,1713.697119,0,0.559198
3416,1257.307940,0,0.652000
6635,1802.415450,1,0.565637
25028,5809.620041,0,0.214343
25387,3735.090280,1,0.342273


In [206]:
# Definindo a função demanda
def demand(x,price,modelo):
    a =model_glm.params[0]
    b1=model_glm.params[1]
    b2=model_glm.params[2]
    return np.exp(a + b1*x + b2*np.log(price))/(1 + np.exp(a + b1*x + b2*np.log(price)))
    #return 1/(1+np.exp(-1*(a + b1*x + b2*np.log(price))))

In [207]:
demanda =demand(0,1713.697119,model_glm)
print('Demanda Base Case: %s' % round(demanda,2))

Demanda Base Case: 0.56


### criando a função para o relatório

In [ ]:
df_relatorio = df_lote_otim.copy()
df_relatorio

In [ ]:
df_relatorio["per_com_pct"] = df_relatorio['per_com']/100
df_relatorio["premio_prob"] = df_relatorio["demanda otimizada"]*df_relatorio["premio"]
df_relatorio["Act_cost_prob"] = df_relatorio["demanda otimizada"]*df_relatorio["Act_cost"]                                                                
df_relatorio["comissao_valor"] = df_relatorio["per_com_pct"] *df_relatorio["premio"]
df_relatorio["comissao_valor_prob"] = df_relatorio["per_com_pct"] *df_relatorio["premio_prob"]


In [ ]:
apolice_ofertada  = np.round(df_relatorio.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio["demanda otimizada"]),0)
premio_ofertado   = np.mean(df_relatorio["premio"])
retencao_estimada = np.mean(df_relatorio["demanda otimizada"])
retencao_base_case = np.mean(df_relatorio["prob_est"])
premio_esperado   = np.sum(df_relatorio["premio_prob"])/np.sum(df_relatorio["demanda otimizada"])
risco_ofertado    = np.mean(df_relatorio["Act_cost"])
risco_esperado    = np.sum(df_relatorio["Act_cost_prob"])/np.sum(df_relatorio["demanda otimizada"])
sp_ofertado       = np.sum(df_relatorio["Act_cost"])/np.sum(df_relatorio["premio"])
sp_esperado       = np.sum(df_relatorio["Act_cost_prob"])/np.sum(df_relatorio["premio_prob"])
comis_ofertado    = np.sum(df_relatorio["comissao_valor"])/np.sum(df_relatorio["premio"])
comis_esperado    = np.sum(df_relatorio["comissao_valor_prob"])/np.sum(df_relatorio["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749

In [ ]:
print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_baseCase : %s' % np.round(retencao_base_case*100,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))